## IBM Data Science Professional Certificate - Capstone Project
#### This notebook will be used in order to present the code my Capstone project

In [1]:
# importing needed libraries
import requests
import numpy as np
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import folium
import json
from pandas import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# fetching data from wikipedia
wiki_data = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wiki_data)

df = pd.read_html(wikipedia_page.content, header=0)[0]
df = df[df.Borough != 'Not assigned']

df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
df.shape

(103, 3)

In [4]:
# check if the Neighborhood column has empty values
n_empty_neighbourhood = df[df['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighbourhood column is empty: {}'.format(n_empty_neighbourhood))

Number of rows on which Neighbourhood column is empty: 0


In [5]:
df = df.rename(columns = {'Postal Code':'PostalCode'})

In [6]:
df_final = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))
df_final = df.reset_index()
df_final.head(10)

,index,PostalCode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df_final = df_final.drop(columns=['index'])
df_final.shape

(103, 3)

In [8]:
# reading coordinates
url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)
df_geo.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
df_geo.shape

(103, 3)

In [10]:
df_final = df_final.join(df_geo.set_index('Postal Code'), on='PostalCode')
df_final.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [11]:
# number of Boroughs and Neighbourhoods
print('The dataframe has {} Boroughs and {} Neighbourhoods.'.format(
        len(df_final['Borough'].unique()),
        df_final.shape[0]
    )
)

The dataframe has 10 Boroughs and 103 Neighbourhoods.


In [12]:
# Number of Neighbourhoods by Borough
df_final.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

In [13]:
df_toronto = df_final[df_final['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto = df_toronto.drop('index', axis=1)
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
df_toronto.shape

(39, 5)

In [15]:
#list with the Boroughs
boroughs = df_toronto['Borough'].unique().tolist()

In [16]:
# coloring each Borough
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto

In [19]:
CLIENT_ID = 'QUTUXBPJJA0U4NRTXQLI0DOB03Y3M434CYEILYEDYG4P3IS1' # your Foursquare ID
CLIENT_SECRET = 'LKZVFNUJLER5SHDOFRKA5R10Z2ST45SWEDUKIETPPYFPTSV3' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QUTUXBPJJA0U4NRTXQLI0DOB03Y3M434CYEILYEDYG4P3IS1
CLIENT_SECRET:LKZVFNUJLER5SHDOFRKA5R10Z2ST45SWEDUKIETPPYFPTSV3


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1588, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [23]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,54,54,54,54,54,54
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,15,15,15,15,15,15
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [24]:
toronto_venues['Venue Category'].unique()[:100]

array(['Coffee Shop', 'Bakery', 'Distribution Center', 'Spa',
       'Restaurant', 'Pub', 'Breakfast Spot', 'Park',
       'Gym / Fitness Center', 'Historic Site', 'Farmers Market',
       'Chocolate Shop', 'Performing Arts Venue', 'French Restaurant',
       'Dessert Shop', 'Yoga Studio', 'Café', 'Theater', 'Event Space',
       'Shoe Store', 'Mexican Restaurant', 'Art Gallery',
       'Cosmetics Shop', 'Asian Restaurant', 'Electronics Store',
       'Beer Store', 'Bank', 'Wine Shop', 'Antique Shop',
       'Portuguese Restaurant', 'Italian Restaurant', 'Beer Bar',
       'Creperie', 'Sushi Restaurant', 'Diner', 'Burrito Place',
       'Fried Chicken Joint', 'Japanese Restaurant', 'Smoothie Shop',
       'Sandwich Place', 'Gym', 'College Auditorium', 'Bar',
       'Clothing Store', 'Pizza Place', 'Comic Shop', 'Plaza',
       'Burger Joint', 'Music Venue', 'Ramen Restaurant', 'Movie Theater',
       'Gastropub', 'Tanning Salon', 'Steakhouse', 'Fast Food Restaurant',
       'Tea Room',

In [25]:
"Greek Restaurant" in toronto_venues['Venue Category'].unique()

True

In [26]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhoods'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1588, 228)


,Neighbourhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_grouped = toronto_onehot.groupby('Neighbourhoods').mean().reset_index()
toronto_grouped.head()

,Neighbourhoods,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.055556
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0,0.016129


In [28]:
len(toronto_grouped[toronto_grouped["Greek Restaurant"] > 0])

6

In [29]:
greek_restaurants = toronto_grouped[["Neighbourhoods","Greek Restaurant"]]

In [30]:
greek_restaurants

,Neighbourhoods,Greek Restaurant
0,Berczy Park,0.018519
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.000000
5,Christie,0.000000
6,Church and Wellesley,0.000000
7,"Commerce Court, Victoria Hotel",0.000000
8,Davisville,0.028571
9,Davisville North,0.000000


In [31]:
from sklearn.cluster import KMeans

kclusters = 3
toronto_grouped_clustering = greek_restaurants.drop('Neighbourhoods', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:20]

array([2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
      dtype=int32)

In [32]:
toronto_merged = greek_restaurants.copy()
columns = ['Neighbourhood']
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhoods']

# adding clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhoods')

toronto_merged.head()

,Neighbourhoods,Greek Restaurant,Cluster Labels
0,Berczy Park,0.018519,2
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.000000,0


In [33]:
toronto_merged = toronto_merged.join(toronto_venues.set_index("Neighbourhood"), on="Neighbourhoods")

print(toronto_merged.shape)
toronto_merged.head()

(1588, 9)


,Neighbourhoods,Greek Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.018519,2,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.018519,2,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.018519,2,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.018519,2,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
0,Berczy Park,0.018519,2,43.644771,-79.373306,Berczy Park,43.648048,-79.375172,Park


In [34]:
toronto_merged = toronto_merged.sort_values(["Cluster Labels"])
toronto_merged.head()

,Neighbourhoods,Greek Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.0,0,43.653206,-79.400049,Saigon Lotus Restaurant,43.654311,-79.399225,Vietnamese Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,The Fifth & Terrace,43.649250,-79.389320,Modern European Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Poke Guys,43.654895,-79.385052,Poke Place
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Astarté Fresh Yogurt Bar,43.647596,-79.386419,Café
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop


In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(toronto_merged['Neighbourhood Latitude'], toronto_merged['Neighbourhood Longitude'], toronto_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
#Cluster 0
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==0) & (toronto_merged['Venue Category'] == 'Greek Restaurant')]

,Neighbourhoods,Greek Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
37,"Toronto Dominion Centre, Design Exchange",0.01,0,43.647177,-79.381576,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
11,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant


In [37]:
#Cluster 1
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==1) & (toronto_merged['Venue Category'] == 'Greek Restaurant')]

,Neighbourhoods,Greek Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Athen's Pastries,43.678166,-79.348927,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Alexandros,43.678304,-79.349486,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Christina's On The Danforth,43.678240,-79.349185,Greek Restaurant
36,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Astoria Shish Kebob House,43.677596,-79.351738,Greek Restaurant


In [38]:
#Cluster 2
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==2) & (toronto_merged['Venue Category'] == 'Greek Restaurant')]

,Neighbourhoods,Greek Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.018519,2,43.644771,-79.373306,Alexandro's World Famous Gyros,43.641663,-79.375214,Greek Restaurant
19,"Little Portugal, Trinity",0.023810,2,43.647927,-79.419750,Mamakas Taverna,43.645908,-79.419654,Greek Restaurant
8,Davisville,0.028571,2,43.704324,-79.388790,souvlaki express,43.707378,-79.389848,Greek Restaurant


## Results

#### Most of the Greek restaurants are in cluster 1 which is found in the areas of The Danforth West and Riverdale

#### The clusters 0 and 2 seems to have the same amount of Greek Restaurants and are located in the areas of Berczy Park, Little Portugal, Trinity, Davisville, Toronto Dominion Centre, Design Exchange, First Canadian Place and Underground city.

#### Altough more detail is needed, such as distance of location from other venues and trasnportation means or GDP of residents per neighbourhood, the recommendation to the union is that places around clustes 0 and 2 are more suitable to open an authentic Greek restaurant.